In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import random
import re
import pandas as pd 

In [ ]:
# WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 스크롤 로직
def scroll_to_bottom(scrollable_element):
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
    while True:
        driver.execute_script("arguments[0].scrollTop += 600;", scrollable_element)
        time.sleep(1)
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
        if new_height == last_height:
            break
        last_height = new_height

# 네이버 지도 접속
driver.get("https://map.naver.com/")
time.sleep(3)

# 검색어 입력 및 검색
search_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
)
search_box.send_keys("성북구 맛집")
search_box.send_keys(Keys.ENTER)
time.sleep(5)

# iframe 전환
try:
    iframe = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
    )
    driver.switch_to.frame(iframe)
except Exception as e:
    print(f"iframe 전환 오류: {e}")
    driver.quit()
    exit()

# 결과 리스트 초기화
results = []
visited_urls = set()
page_number = 1
MAX_PAGES = 5  # 최대 페이지 수
MAX_RESULTS = 150  # 최대 결과 수 제한

while page_number <= MAX_PAGES and len(results) < MAX_RESULTS:
    try:
        # 스크롤을 맨 아래까지 내리기
        scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")
        scroll_to_bottom(scrollable_element)

        # 현재 페이지의 가게 리스트 가져오기
        store_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "place_bluelink"))
        )
        print(f"현재 {page_number}페이지) 총 {len(store_elements)}개의 가게를 찾았습니다.")

        for store in store_elements:
            if len(results) >= MAX_RESULTS:
                print("최대 결과 수에 도달하여 크롤링을 종료합니다.")
                break

            try:
                store.click()
                time.sleep(random.uniform(3, 5))

                # 상세 iframe 전환
                driver.switch_to.default_content()
                detail_iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#entryIframe"))
                )
                driver.switch_to.frame(detail_iframe)

                # 페이지 소스 파싱
                html = driver.page_source
                soup = BeautifulSoup(html, "lxml")

                # 가게 정보 추출
                place_name = soup.select_one("span.GHAhO").text.strip() if soup.select_one("span.GHAhO") else "N/A"
                category = soup.select_one("span.lnJFt").text.strip() if soup.select_one("span.lnJFt") else "N/A"
                address = soup.select_one("span.LDgIH").text.strip() if soup.select_one("span.LDgIH") else "N/A"
                current_url = driver.current_url
                res_code = re.findall(r"place/(\d+)", current_url)
                store_url = f'https://pcmap.place.naver.com/restaurant/{res_code[0]}/review/visitor#' if res_code else "N/A"

                # 중복 여부 확인
                if store_url not in visited_urls:
                    visited_urls.add(store_url)
                    results.append({
                        "가게 이름": place_name,
                        "업종구분": category,
                        "주소": address,
                        "URL": store_url
                    })
                    print(f"{len(results)}. {place_name}")

                # iframe 복귀
                driver.switch_to.default_content()
                iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
                )
                driver.switch_to.frame(iframe)

            except Exception as e:
                print(f"가게 크롤링 중 오류 발생: {e}")

        if len(results) >= MAX_RESULTS:
            print("최대 결과 수에 도달하여 크롤링을 종료합니다.")
            break

        # '다음' 버튼 클릭
        if page_number < MAX_PAGES:
            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//a[@class="eUTV2" and @aria-disabled="false"]'))
                )
                driver.execute_script("arguments[0].click();", next_button)
                print(f"'다음' 버튼 클릭: 페이지 {page_number + 1}로 이동 중")
                time.sleep(random.uniform(5, 10))

                # iframe 전환
                driver.switch_to.default_content()
                iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
                )
                driver.switch_to.frame(iframe)

            except Exception as e:
                print(f"'다음' 버튼 클릭 중 오류 발생: {e}")
                break

        page_number += 1

    except Exception as e:
        print(f"페이지 크롤링 중 오류 발생: {e}")
        break

file_name = "Seongbuk-gu_restaurants.csv"
df = pd.DataFrame(results)
df.to_csv(file_name, index=False, encoding="utf-8-sig")

print(f"크롤링 완료. 결과는 {file_name}에 저장되었습니다.")
driver.quit()

현재 1페이지) 총 76개의 가게를 찾았습니다.
1. 광명대창집 성신여대점
2. 비양도 본점
3. 남해바다마차
4. 온오프 성신여대점
5. 팔백집 성신여대본점
6. 수아당
7. 김태완스시 성신여대점
8. 광명수산
9. 경성고기꾼 성신여대점
10. 김통
11. 문화식당
12. 영순관
13. 나폴레옹과자점 본점
14. 별관 성신여대
15. 플랜터247
16. 멘지 성신여대점
17. 해탄 미아사거리점
18. 애정마라샹궈
19. 자연에서왔소 본점
20. 커피명가 안암점
21. 한술식당 길음본점
22. 양슐랭 성신여대점
23. 일번지정육식당 돌곶이점
24. 뮈에
25. 르부아
26. 르한스
27. 밀곳간
28. 유메오뎅 성신여대
29. 동경산책 성신여대점
30. 꿀단지 성신여대점
31. 샤브온당 성신여대본점
32. 공푸
33. 뿡어당 성신여대점
34. 연리지
35. 금뚜껑
36. 고석환손만두전골
37. 창창서울
38. 팔백집
39. 서울노마드
40. 떡고물즉석떡볶이전문점한성대점
41. 두껍다회선생
42. 백소정 하월곡점
43. 선류도
44. 멘야하나비 성신여대점
45. 츠루츠루 마제소바
46. 우정초밥
47. 벤베누토
48. liike coffee
49. 구룡포 계절 회집
50. 성북동빵공장
51. 밀양손칼국수
52. 유영
53. 비나케이크
54. 봄담아 하월곡점
55. 무르무르드구스토
56. 이공김밥 안암본점
57. 문화식당
58. 고대 어시장
59. 운치
60. 성북동누룽지백숙
61. 긴자료코 성신여대점
62. 밥짓고티우림
63. 부산꼼장어
64. 샛길무드 성신여대점
65. 한원
66. 올쓰
67. 성술
68. 하남돼지집 정릉점
69. 타코아리 타코야끼 성신여대점
70. 숙수도가
71. 89번가 1호점
72. 제순식당 성신여대점
73. 한술식당 고대안암점
74. 태조감자국
75. 동선식당 성신여대점
'다음' 버튼 클릭: 페이지 2로 이동 중
현재 2페이지) 총 76개의 가게를 찾았습니다.
76. 와우곱창 성신여대점
77. 핵밥 성신여대본점
78. 동동 냉면 돈까스


가게 크롤링 중 오류 발생: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=131.0.6778.108); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x003B3433+25059]
	(No symbol) [0x0033CE34]
	(No symbol) [0x0021BEC3]
	(No symbol) [0x002219D1]
	(No symbol) [0x002238EA]
	(No symbol) [0x00223967]
	(No symbol) [0x00261832]
	(No symbol) [0x00255EFF]
	(No symbol) [0x00281EFC]
	(No symbol) [0x00255A24]
	(No symbol) [0x00282194]
	(No symbol) [0x0029B51E]
	(No symbol) [0x00281C96]
	(No symbol) [0x00253FAC]
	(No symbol) [0x00254F3D]
	GetHandleVerifier [0x006A5593+3113795]
	GetHandleVerifier [0x006BA25A+3198986]
	GetHandleVerifier [0x006B2A32+3168226]
	GetHandleVerifier [0x004532A0+680016]
	(No symbol) [0x0034577D]
	(No symbol) [0x00342A28]
	(No symbol) [0x00342BC5]
	(No symbol) [0x00335820]
	BaseThreadInitThunk [0x75E3FCC9+2

가게 크롤링 중 오류 발생: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=131.0.6778.108); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x003B3433+25059]
	(No symbol) [0x0033CE34]
	(No symbol) [0x0021BEC3]
	(No symbol) [0x002219D1]
	(No symbol) [0x002238EA]
	(No symbol) [0x00223967]
	(No symbol) [0x00261832]
	(No symbol) [0x00255EFF]
	(No symbol) [0x00281EFC]
	(No symbol) [0x00255A24]
	(No symbol) [0x00282194]
	(No symbol) [0x0029B51E]
	(No symbol) [0x00281C96]
	(No symbol) [0x00253FAC]
	(No symbol) [0x00254F3D]
	GetHandleVerifier [0x006A5593+3113795]
	GetHandleVerifier [0x006BA25A+3198986]
	GetHandleVerifier [0x006B2A32+3168226]
	GetHandleVerifier [0x004532A0+680016]
	(No symbol) [0x0034577D]
	(No symbol) [0x00342A28]
	(No symbol) [0x00342BC5]
	(No symbol) [0x00335820]
	BaseThreadInitThunk [0x75E3FCC9+2

'다음' 버튼 클릭: 페이지 4로 이동 중
현재 4페이지) 총 70개의 가게를 찾았습니다.
150. S서서갈비
최대 결과 수에 도달하여 크롤링을 종료합니다.
최대 결과 수에 도달하여 크롤링을 종료합니다.
크롤링 완료. 결과는 Seongbuk-gu_Navermap_restaurants.csv에 저장되었습니다.
